In [1]:
import pandas as pd
import pickle
import requests

In [2]:
df = pd.read_csv('../data/elixier_physics_all.csv', index_col=0)

In [17]:
df['schlagwörter']

0       Astronomie Planet Sekundarstufe I Sekundarstuf...
1       Physik Astronomie Planet Himmelskörper Sekunda...
2       Physik Elektrizität Sekundarstufe I Elektrizit...
3                     Physik Wellenlehre Sekundarstufe II
4                                     Lernsoftware Physik
                              ...                        
2894    Zeitschrift Forscher Forschung Müll Abfall Mee...
2895    Deutsch als Zweitsprache Deutsch als Fremdspra...
2896    Zeitschrift Forscher Forschung Stadt Nahrung L...
2897                         Ökologie Grundschule Technik
2898                                           Lehrmittel
Name: schlagwörter, Length: 2899, dtype: object

In [3]:
na_counts = [(df[df[column].isna()].shape[0] / df.shape[0], column) for column in df.columns]

for pct_na, column in sorted(na_counts):
    print("%.2f%% N/a for %s" % (pct_na * 100, column))

0.00% N/a for is_pdf
0.00% N/a for position
0.00% N/a for themenbereich
0.00% N/a for title
0.00% N/a for url
0.10% N/a for geeignet für
0.21% N/a for sprache
2.17% N/a for domain
2.45% N/a for description
5.45% N/a for lizenz
14.83% N/a for schlagwörter
23.97% N/a for freie schlagwörter
28.22% N/a for bildungsebene
42.12% N/a for autor
45.15% N/a for lernressourcentyp
56.26% N/a for anbieter
62.68% N/a for mindestalter
62.75% N/a for höchstalter
87.41% N/a for lange beschreibung
98.17% N/a for beschreibung für schüler
98.55% N/a for titel für schüler
98.62% N/a for ortsbezüge
99.45% N/a for null
99.90% N/a for herausgeber
99.97% N/a for kompletter titel


In [21]:
df['url'].nunique() / df.shape[0]

1.0

In [4]:
df.lernressourcentyp.apply(lambda l: [w.strip().lower() for w in l.split(';')] if type(l) == str else []).apply(pd.Series).stack().reset_index(drop=True).value_counts().iloc[:50]

arbeitsblatt                                           606
unterrichtsplanung                                     428
text                                                   374
audiovisuelles medium                                  357
website                                                218
simulation                                             102
aufgabe                                                 80
grafik                                                  67
interaktives material                                   54
primärmaterial/quelle                                   50
arbeitsmaterial                                         47
lernkontrolle                                           32
portal                                                  21
select: lernkontrolle                                   21
projekt                                                 16
anwendung/software                                      14
bild/grafik                                             

In [5]:
df['geeignet für']

0       Lehrer; Schüler
1       Lehrer; Schüler
2       Lehrer; Schüler
3       Lehrer; Schüler
4       Lehrer; Schüler
             ...       
2894            Schüler
2895    Schüler; Lehrer
2896            Schüler
2897    Lehrer; Schüler
2898    Lehrer; Schüler
Name: geeignet für, Length: 2899, dtype: object

In [6]:
df['age_range'] = list(zip(df.mindestalter, df['höchstalter']))

In [7]:
df.age_range.value_counts()[:10]

(nan, nan)                                1817
(6, 15)                                    244
(10, 18)                                   210
(16, 18)                                    94
(10, 15)                                    90
(Lehrer Online: 10, Lehrer Online: 18)      53
(6, 9)                                      41
(6, 18)                                     40
(22, 22)                                    32
(12, 16)                                    28
Name: age_range, dtype: int64

In [8]:
import numpy as np

In [9]:
df[df.age_range != (np.nan, np.nan)].shape[0] / df.shape[0]

0.3732321490169024

In [10]:
df[df.age_range.apply(lambda x: bool(x[0] and x[1]))].shape[0] / df.shape[0]

1.0

In [11]:
df.bildungsebene

0       Sekundarstufe I Sekundarstufe II
1       Sekundarstufe I Sekundarstufe II
2                        Sekundarstufe I
3       Sekundarstufe I Sekundarstufe II
4       Sekundarstufe I Sekundarstufe II
                      ...               
2894         Primarstufe Sekundarstufe I
2895         Primarstufe Sekundarstufe I
2896         Primarstufe Sekundarstufe I
2897    Sekundarstufe I Sekundarstufe II
2898    Sekundarstufe I Sekundarstufe II
Name: bildungsebene, Length: 2899, dtype: object

In [12]:
bildungsebene = [be.lower() for be in [
    "Sekundarstufe I", # (1645)
    "Sekundarstufe II", # (1118)
    "Primarstufe", # (426)
    "Hochschule", # (85)
    "Berufliche Bildung", # (68)
    "Elementarbildung", # (28)
    "Fort- und Weiterbildung" # (27)
]]

In [13]:
bildungsebene_clean = []
for val in df.bildungsebene:
    if type(val) == str:
        str_out = []
        val_lower = val.lower()
        for be in bildungsebene:
            if be in val_lower:
                str_out.append(be)
        bildungsebene_clean.append(str_out)
    else:
        bildungsebene_clean.append([])
df['bildungsebene_clean'] = bildungsebene_clean

In [14]:
df.bildungsebene_clean.apply(pd.Series).stack().reset_index(drop=True).value_counts()

sekundarstufe i            2002
sekundarstufe ii           1109
primarstufe                 422
hochschule                   80
berufliche bildung           67
fort- und weiterbildung      27
elementarbildung             23
dtype: int64

In [35]:
df.is_pdf.value_counts()

False    2729
True      170
Name: is_pdf, dtype: int64

In [36]:
df[~df.is_pdf].domain.value_counts()

www.leifiphysik.de            821
www.lehrer-online.de          636
www.supra-lernplattform.de     92
www.planet-schule.de           74
www.bildung-lsa.de             72
                             ... 
www.hyaden.de                   1
www2.hu-berlin.de               1
www.agcom.uni-bremen.de         1
supermint.de                    1
vimeo.com                       1
Name: domain, Length: 344, dtype: int64

In [10]:
for url in df[df.url.str.contains('supra-lernplattform')].url[:5]:
    print(url)

http://www.supra-lernplattform.de/index.php/lernfeld-natur-und-technik/verbrennung/einheit-1-feuer
http://www.supra-lernplattform.de/index.php/lernfeld-natur-und-technik/warm-kalt/warm-kalt-zusatzangebot
http://www.supra-lernplattform.de/index.php/lernfeld-natur-und-technik/spiegel/einheit-6-spiegelsymmetrie
http://www.supra-lernplattform.de/index.php/lernfeld-natur-und-technik/schall/einheit-11-aufbau-und-funktion-des-ohres
http://www.supra-lernplattform.de/index.php/lernfeld-natur-und-technik/verbrennung/sachinformationen-fuer-die-lehrkraft


In [11]:
for url in df[df.url.str.contains('planet-schule.de')].url[:5]:
    print(url)

https://www.planet-schule.de/wissenspool/total-phaenomenal-technik/inhalt/unterricht/strahlen-schaffen-durchblick.html
http://www.planet-schule.de/sf/multimedia-simulationen-detail.php?projekt=blindenschrift
http://www.planet-schule.de/wissenspool/achtung-experiment/inhalt/2-sendungsstaffel/der-begehbare-regenbogen.html
http://www.planet-schule.de/sf/multimedia-simulationen-detail.php?projekt=hebelwirkung
http://www.planet-schule.de/sf/multimedia-lernspiele-detail.php?projekt=raumschiff_stargaze


In [39]:
with open('../data/responses', 'rb') as f:
    responses = pickle.load(f)

In [42]:
len(responses)

2899

In [43]:
df['responses'] = responses

In [45]:
type(df.responses.iloc[0])

requests.models.Response

In [57]:
successul_responses = df[df.responses.apply(lambda r: type(r) == requests.models.Response)]
successul_responses = successul_responses[successul_responses.responses.apply(lambda r: r.status_code) == 200]

In [71]:
successul_responses['content_type'] = successul_responses.responses.apply(lambda r: r.headers['Content-Type']).\
    apply(lambda ct: ct.lower().replace('charset=utf-8', '').strip().replace(';', '').strip())

In [72]:
successul_responses.shape[0] / df.shape[0]

0.9603311486719558

In [74]:
successul_responses.content_type.value_counts()

text/html                                                                  2530
application/pdf                                                             160
audio/mpeg                                                                   29
text/html charset=iso-8859-1                                                 28
application/vnd.openxmlformats-officedocument.wordprocessingml.document      14
application/zip                                                              13
application/octet-stream                                                      3
application/x-shockwave-flash                                                 2
image/jpeg                                                                    2
application/x-msdos-program                                                   1
text/htmlcharset=iso-8859-1                                                   1
image/gif                                                                     1
Name: content_type, dtype: int64

In [77]:
successul_responses[successul_responses.content_type == 'text/html'].domain.value_counts().iloc[:50]

www.leifiphysik.de                        821
www.lehrer-online.de                      636
www.supra-lernplattform.de                 92
www.bildung-lsa.de                         80
www.planet-schule.de                       57
de.wikipedia.org                           55
www.abi-physik.de                          53
www.zum.de                                 33
www.komm-mach-mint.de                      26
www.heise.de                               26
digitallearninglab.de                      21
www.dlr.de                                 20
primas.ph-freiburg.de                      17
www.science-on-stage.de                    16
www.esa.int                                16
www.zauberhafte-physik.net                 16
www.wissenschaftsjahr.de                   15
lernarchiv.bildung.hessen.de               13
blogs.helmholtz.de                         13
www.bildungsserver.de                      10
www.wissenschaft-schulen.de                 9
physikaufgaben.de                 

In [87]:
leifi = successul_responses[successul_responses.domain == 'www.leifiphysik.de']
print(leifi[leifi.bildungsebene.isna()].shape[0] / leifi.shape[0])

0.9926918392204629


In [96]:
from trafilatura.spider import focused_crawler

In [95]:
for url in successul_responses[successul_responses.content_type == 'text/html'].url:
    focused_crawler()

0           http://www.lehrer-online.de/neptun-uranus.php
1       http://www.lehrer-online.de/venustransit-2004.php
2       http://www.lehrer-online.de/virtuelle-elektriz...
3             http://www.lehrer-online.de/wellenwanne.php
4                  http://www.lehrer-online.de/physix.php
                              ...                        
2890    https://wissenschaftsjahr-2020.visionkino.de/2...
2891    https://wissenschaftsjahr-2020.visionkino.de/a...
2893    https://platform.govie.de/share/Z8qYGMWSRwiDAm...
2897     http://www.lehrer-online.de/energiespiel2010.php
2898    http://www.lehrer-online.de/wiimote-whiteboard...
Name: url, Length: 2530, dtype: object